<a href="https://colab.research.google.com/github/mojo46/Vehical_parts_detection/blob/master/Parts_detection_yolov2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 14997, done.
remote: Total 14997 (delta 0), reused 0 (delta 0), pack-reused 14997
Receiving objects: 100% (14997/14997), 13.38 MiB | 22.35 MiB/s, done.
Resolving deltas: 100% (10194/10194), done.


Chnage to Darknet Directory

In [ ]:
import os
os.chdir('/content/darknet')

Clone vehical parts detection git

In [ ]:
!git clone https://github.com/mojo46/Vehical_parts_detection.git

Cloning into 'Vehical_parts_detection'...
remote: Enumerating objects: 742, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 742 (delta 0), reused 0 (delta 0), pack-reused 739
Receiving objects: 100% (742/742), 50.34 MiB | 41.67 MiB/s, done.
Resolving deltas: 100% (111/111), done.


Remove the Exsisting Makefile

In [ ]:
!rm -r /content/darknet/Makefile

Copy the Required Data and files to Darknet folder

In [ ]:
!cp -r /content/darknet/Vehical_parts_detection/* /content/darknet

### Display the Makefile to check weather the gpu is enabled to make the training a crazy faster 

In [ ]:
!cat /content/darknet/Makefile

In [ ]:
%%writefile /content/darknet/Makefile

GPU=1
CUDNN=0
CUDNN_HALF=0
OPENCV=0
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)

DEBUG=0

# -gencode arch=compute_30,code=sm_30 \


ARCH= -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	  -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro RTX 8000, Quadro RTX 6000, Quadro RTX 5000, Tesla T4, XNOR Tensor Cores
# ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# Jetson XAVIER
# ARCH= -gencode arch=compute_72,code=[sm_72,compute_72]

# GTX 1080, GTX 1070, GTX 1060, GTX 1050, GTX 1030, Titan Xp, Tesla P40, Tesla P4
# ARCH= -gencode arch=compute_61,code=sm_61 -gencode arch=compute_61,code=compute_61

# GP100/Tesla P100 - DGX-1
# ARCH= -gencode arch=compute_60,code=sm_60

# For Jetson TX1, Tegra X1, DRIVE CX, DRIVE PX - uncomment:
# ARCH= -gencode arch=compute_53,code=[sm_53,compute_53]

# For Jetson Tx2 or Drive-PX2 uncomment:
# ARCH= -gencode arch=compute_62,code=[sm_62,compute_62]


VPATH=./src/
EXEC=darknet
OBJDIR=./obj/

ifeq ($(LIBSO), 1)
LIBNAMESO=libdarknet.so
APPNAMESO=uselib
endif

CC=gcc
CPP=g++
NVCC=nvcc
OPTS=-Ofast
LDFLAGS= -lm -pthread
COMMON= -Iinclude/ -I3rdparty/stb/include
CFLAGS=-Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC

ifeq ($(DEBUG), 1)
#OPTS= -O0 -g
#OPTS= -Og -g
COMMON+= -DDEBUG
CFLAGS+= -DDEBUG
else
ifeq ($(AVX), 1)
CFLAGS+= -ffp-contract=fast -mavx -mavx2 -msse3 -msse4.1 -msse4.2 -msse4a
endif
endif

CFLAGS+=$(OPTS)

ifeq ($(OPENCV), 1)
COMMON+= -DOPENCV
CFLAGS+= -DOPENCV
LDFLAGS+= `pkg-config --libs opencv`
COMMON+= `pkg-config --cflags opencv`
endif

ifeq ($(OPENMP), 1)
CFLAGS+= -fopenmp
LDFLAGS+= -lgomp
endif

ifeq ($(GPU), 1)
COMMON+= -DGPU -I/usr/local/cuda/include/
CFLAGS+= -DGPU
ifeq ($(OS),Darwin) #MAC
LDFLAGS+= -L/usr/local/cuda/lib -lcuda -lcudart -lcublas -lcurand
else
LDFLAGS+= -L/usr/local/cuda/lib64 -lcuda -lcudart -lcublas -lcurand
endif
endif

ifeq ($(CUDNN), 1)
COMMON+= -DCUDNN
ifeq ($(OS),Darwin) #MAC
CFLAGS+= -DCUDNN -I/usr/local/cuda/include
LDFLAGS+= -L/usr/local/cuda/lib -lcudnn
else
CFLAGS+= -DCUDNN -I/usr/local/cudnn/include
LDFLAGS+= -L/usr/local/cudnn/lib64 -lcudnn
endif
endif

ifeq ($(CUDNN_HALF), 1)
COMMON+= -DCUDNN_HALF
CFLAGS+= -DCUDNN_HALF
ARCH+= -gencode arch=compute_70,code=[sm_70,compute_70]
endif

ifeq ($(ZED_CAMERA), 1)
CFLAGS+= -DZED_STEREO -I/usr/local/zed/include
LDFLAGS+= -L/usr/local/zed/lib -lsl_core -lsl_input -lsl_zed
#-lstdc++ -D_GLIBCXX_USE_CXX11_ABI=0 
endif

OBJ=image_opencv.o http_stream.o gemm.o utils.o dark_cuda.o convolutional_layer.o list.o image.o activations.o im2col.o col2im.o blas.o crop_layer.o dropout_layer.o maxpool_layer.o softmax_layer.o data.o matrix.o network.o connected_layer.o cost_layer.o parser.o option_list.o darknet.o detection_layer.o captcha.o route_layer.o writing.o box.o nightmare.o normalization_layer.o avgpool_layer.o coco.o dice.o yolo.o detector.o layer.o compare.o classifier.o local_layer.o swag.o shortcut_layer.o activation_layer.o rnn_layer.o gru_layer.o rnn.o rnn_vid.o crnn_layer.o demo.o tag.o cifar.o go.o batchnorm_layer.o art.o region_layer.o reorg_layer.o reorg_old_layer.o super.o voxel.o tree.o yolo_layer.o gaussian_yolo_layer.o upsample_layer.o lstm_layer.o conv_lstm_layer.o scale_channels_layer.o sam_layer.o
ifeq ($(GPU), 1) 
LDFLAGS+= -lstdc++ 
OBJ+=convolutional_kernels.o activation_kernels.o im2col_kernels.o col2im_kernels.o blas_kernels.o crop_layer_kernels.o dropout_layer_kernels.o maxpool_layer_kernels.o network_kernels.o avgpool_layer_kernels.o
endif

OBJS = $(addprefix $(OBJDIR), $(OBJ))
DEPS = $(wildcard src/*.h) Makefile include/darknet.h

all: obj backup results setchmod $(EXEC) $(LIBNAMESO) $(APPNAMESO)

ifeq ($(LIBSO), 1)
CFLAGS+= -fPIC

$(LIBNAMESO): $(OBJS) include/yolo_v2_class.hpp src/yolo_v2_class.cpp
	$(CPP) -shared -std=c++11 -fvisibility=hidden -DLIB_EXPORTS $(COMMON) $(CFLAGS) $(OBJS) src/yolo_v2_class.cpp -o $@ $(LDFLAGS)

$(APPNAMESO): $(LIBNAMESO) include/yolo_v2_class.hpp src/yolo_console_dll.cpp
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) -o $@ src/yolo_console_dll.cpp $(LDFLAGS) -L ./ -l:$(LIBNAMESO)
endif

$(EXEC): $(OBJS)
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) $^ -o $@ $(LDFLAGS)

$(OBJDIR)%.o: %.c $(DEPS)
	$(CC) $(COMMON) $(CFLAGS) -c $< -o $@

$(OBJDIR)%.o: %.cpp $(DEPS)
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) -c $< -o $@

$(OBJDIR)%.o: %.cu $(DEPS)
	$(NVCC) $(ARCH) $(COMMON) --compiler-options "$(CFLAGS)" -c $< -o $@

obj:
	mkdir -p obj
backup:
	mkdir -p backup
results:
	mkdir -p results
setchmod:
	chmod +x *.sh

.PHONY: clean

clean:
	rm -rf $(OBJS) $(EXEC) $(LIBNAMESO) $(APPNAMESO)

Overwriting /content/darknet/Makefile


Compile the make file to generate darknet executable

In [ ]:
!make

In [ ]:
!nvcc --version

In [ ]:
! ./darknet

usage: ./darknet <function>


Load the pretrained model for easy training your use model

In [ ]:
!wget https://pjreddie.com/media/files/darknet19_448.conv.23

--2021-05-11 05:04:44--  https://pjreddie.com/media/files/darknet19_448.conv.23
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79327120 (76M) [application/octet-stream]
Saving to: ‘darknet19_448.conv.23.1’

darknet19_448.conv. 100%[===================>]  75.65M  37.3MB/s    in 2.0s    

2021-05-11 05:04:46 (37.3 MB/s) - ‘darknet19_448.conv.23.1’ saved [79327120/79327120]



Start training ,
after 100 epoch's u can find weights under baskup folder

In [ ]:
!./darknet detector train /content/darknet/obj.data /content/darknet/yolo-voc.2.0.cfg darknet19_448.conv.23 -dont_show 0

Streaming output truncated to the last 5000 lines.
Region Avg IOU: 0.661385, Class: 0.990156, Obj: 0.583845, No Obj: 0.009208, Avg Recall: 0.807692,  count: 52
Region Avg IOU: 0.730498, Class: 0.991165, Obj: 0.699511, No Obj: 0.009177, Avg Recall: 0.875000,  count: 40
Region Avg IOU: 0.752847, Class: 0.999556, Obj: 0.724981, No Obj: 0.009101, Avg Recall: 0.976191,  count: 42
Region Avg IOU: 0.725018, Class: 0.999273, Obj: 0.707408, No Obj: 0.008552, Avg Recall: 0.883721,  count: 43
Region Avg IOU: 0.675274, Class: 0.998816, Obj: 0.655890, No Obj: 0.007970, Avg Recall: 0.871795,  count: 39
Region Avg IOU: 0.630577, Class: 0.950092, Obj: 0.612673, No Obj: 0.009310, Avg Recall: 0.796296,  count: 54
Region Avg IOU: 0.761170, Class: 0.999219, Obj: 0.681837, No Obj: 0.010219, Avg Recall: 0.958333,  count: 48

 835: 0.358142, 0.415431 avg loss, 0.001000 rate, 3.929739 seconds, 53440 images, 48.454373 hours left
Loaded: 0.000044 seconds
Region Avg IOU: 0.712839, Class: 0.994635, Obj: 0.660993,

### Test the weather make has generated darknet Executable 

Either download the weights OR Mount your Google Drive and save the weight.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir '/content/drive/My Drive/Vehical_parts'

In [ ]:
!cp -r '/content/darknet/backup' '/content/drive/My Drive/Vehical_parts'

Test the model with a test image and get find the labled image as predictions.png

In [ ]:
!./darknet detector test.txt /content/darknet/obj.data /content/darknet/yolo-voc.2.0.cfg darknet19_448.conv.23 -dont_show 0

/bin/bash: ./darknet: No such file or directory


In [ ]:
!cp '/content/drive/My Drive/Vehical_parts/backup/yolo-voc_last.weights' '/content/darknet/backup'

In [ ]:
!./darknet detector test /content/darknet/obj.data /content/darknet/yolo-voc.2.0.cfg /content/darknet/backup/yolo-voc_last.weights '/content/darknet/download (1).jpg'

/bin/bash: ./darknet: No such file or directory
